<a href="https://colab.research.google.com/github/aartikushal/IC-LLAMA-ASSIGNMENT/blob/main/LLAMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1) Install dependencies**

In [1]:
!pip install transformers torch accelerate

**2) Login to Hugging Face**

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf" # meta-llama/Llama-2-7b-hf

tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1001: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Device set to use cuda:0


**4) Chat helper function**

In [ ]:
def generate_llama_response(prompt: str) -> None:
    """
    Generate a response from the Llama model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """
    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=256,
    )
    print("Chatbot:", sequences[0]['generated_text'])



prompt = 'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n'
generate_llama_response(prompt)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Chatbot: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

You might enjoy "The Sopranos", "The Wire", "Mad Men", and "Narcos". These shows are similar to "Breaking Bad" and "Band of Brothers" in terms of their gritty realism, complex characters, and compelling storylines. "The Sopranos" is a crime drama that explores the life of a New Jersey mob boss, while "The Wire" is a gritty drama that examines the drug trade in Baltimore from multiple perspectives. "Mad Men" is a period drama that follows the lives of advertising executives in the 1960s, and "Narcos" is a crime drama that tells the true story of the rise and fall of Colombian drug kingpin Pablo Escobar. All of these shows have received critical acclaim and have a dedicated fan base, so I hope you enjoy them!


**5) Quick test**

In [ ]:
prompt = """I'm a programmer and Python is my favorite language because of it's simple syntax and variety of applications I can build with it.\
Based on that, what language should I learn next?\
Give me 5 recommendations"""
generate_llama_response(prompt)

Chatbot: I'm a programmer and Python is my favorite language because of it's simple syntax and variety of applications I can build with it.Based on that, what language should I learn next?Give me 5 recommendations for programming languages that are similar to Python and have a wide range of applications.

There are several programming languages that are similar to Python and have a wide range of applications. Here are five recommendations:

1. JavaScript: JavaScript is a popular language for web development and is used by most websites for client-side scripting. It's also used in mobile app development, game development, and server-side programming with technologies like Node.js. JavaScript has a syntax similar to Python and is known for its flexibility and versatility.
2. Ruby: Ruby is a dynamic language that's known for its simplicity and readability. It's often used for web development with the Ruby on Rails framework, but it's also used in scripting, system administration, and data

In [ ]:
prompt = 'How to do quick breakfast using non startchy veggies?\n'
generate_llama_response(prompt)

Chatbot: How to do quick breakfast using non startchy veggies?

I'm looking for some quick and easy breakfast ideas using non-starchy vegetables. Here are some options I've come up with so far:

1. Spinach and feta omelette: Whisk together eggs, spinach, and feta cheese, and cook in a skillet until the eggs are set.
2. Avocado toast: Toast some whole grain bread, mash up an avocado, and spread it on top. Add a sliced egg or some tomato if you like.
3. Roasted vegetable salad: Roast some vegetables like broccoli, cauliflower, or Brussels sprouts, and toss them with some greens and a simple vinaigrette.
4. Smoothie bowl: Blend together your favorite non-starchy vegetables with some fruit and milk, and top with granola and fruit.
5. Veggie-packed quiche: Whisk together eggs, vegetables like bell peppers, onions, and mushrooms, and bake in a pie crust until set.


In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["bye", "quit", "exit"]:
        print("Chatbot: Goodbye!")
        break
    generate_llama_response(user_input)


You: who is pm of indoa
Chatbot: who is pm of indoa

Who is the Prime Minister of India?

The Prime Minister of India is Narendra Modi. He was born on September 17, 1950, in Vadnagar, Gujarat, India. Modi is a member of the Bharatiya Janata Party (BJP) and has been the Prime Minister of India since May 2014, when he was elected to a first term in office. He was re-elected in the 2019 Indian general election and is currently serving his second term as Prime Minister.

Modi has been a key figure in Indian politics for many years, serving as the Chief Minister of Gujarat from 2001 to 2014. He has been credited with transforming Gujarat's economy and infrastructure during his tenure as Chief Minister.

As Prime Minister, Modi has implemented several policies and initiatives aimed at promoting economic growth, improving governance, and increasing access to basic services such as healthcare and education. Some of his notable initiatives include the demonetization of high-denomination currenc

**6) Gradio Chat UI**